In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mdates
import numpy as np

# Load the MT5 CSV
df = pd.read_csv( "xauusdm15.filtered.csv" )

# Set DateTime as index
df['DateTime'] = pd.to_datetime(df["DateTime"])
df.set_index("DateTime", inplace=True)

# Optional: keep only OHLC + Volume for plotting
ohlc_df = df[["Open", "High", "Low", "Close", "Volume"]]


In [ ]:
def add_vertical_lines_on_flagged_candles(axis, df, flag_col='isBear', color='red', linewidth=2):
    ax= axis[0]
    """
    Add full-plot-height vertical lines on flagged candles to an existing plot.
    """
    ylim = ax.get_ylim()  # Get current plot limits
    for i, flag in enumerate(df[flag_col]):
        if flag == 1:
            ax.vlines(i, ylim[0], ylim[1], color=color, linewidth=linewidth)
    ax.figure.canvas.draw()  # Update the figure



In [ ]:
import pandas as pd

def detect_macd_cross(df, macd_col='MACD', signal_col='MACD_Signal'):
    """
    Detects MACD crossovers in a DataFrame.

    Parameters:
    - df: DataFrame containing MACD and Signal columns
    - macd_col: name of the MACD column
    - signal_col: name of the Signal column

    Returns:
    - A Series with 1 (buy), -1 (sell), 0 (no crossover)
    """
    cross = pd.Series(0, index=df.index)

    # Buy signal: MACD crosses above Signal
    buy_mask = (df[macd_col].shift(1) < df[signal_col].shift(1)) & (df[macd_col] > df[signal_col])
    cross[buy_mask] = 1

    # Sell signal: MACD crosses below Signal
    sell_mask = (df[macd_col].shift(1) > df[signal_col].shift(1)) & (df[macd_col] < df[signal_col])
    cross[sell_mask] = -1

    return cross


In [ ]:
import talib
import pandas as pd

# Assume ohlc_df has a 'Close' column
# MACD calculation
macd_line, signal_line, macd_hist = talib.MACD(
    ohlc_df['Close'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

# Add to your DataFrame
ohlc_df['MACD'] = macd_line
ohlc_df['MACD_Signal'] = signal_line
ohlc_df['MACD_Hist'] = macd_hist

# View the first few rows
ohlc_df['Crossover'] = detect_macd_cross(ohlc_df)
ohlc_df[['MACD', 'MACD_Signal', 'MACD_Hist', 'Crossover']].tail()


In [ ]:
import mplfinance as mpf
import pandas as pd

def plot_candles(df):
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    addplots = []

    # Only add EMA if it exists
    # addplots.append(mpf.make_addplot(df['EMA_51'], color='orange', width=1.2))

    addplots.append(mpf.make_addplot(df['MACD'], panel=1, color='green', width=1.2))
    addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=1, color='red', width=1.2))

    charttype = "candle"
    title = f"{charttype.capitalize()} - Chart"

    fig, axes = mpf.plot(
        df,
        type=charttype,
        style='classic',
        volume=False,
        figratio=(14, 7),
        figscale=1.2,
        returnfig=True,
        addplot=addplots,
        title=title
    )
    return fig, axes


In [ ]:
plot_candles(ohlc_df.iloc[23400:]);